In [14]:
import numpy as np

grid_size = (5,5,5)
n_points = np.prod(grid_size)
params = np.zeros(np.prod(grid_size)*3)
for i, p in enumerate(np.ndindex(grid_size)):
    params[i] = p[2]
    params[n_points + i] = p[1]
    params[2*n_points + i] = p[0]

(375,)